In [10]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="phpipamadmin",
  database="assetdb"
)

mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES")
for x in mycursor:
    print(x)
mycursor.close()

('assetdb_main',)
('p3_netflow',)


True

In [2]:
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE p_netflow (srcaddr INT UNSIGNED, srcport MEDIUMINT UNSIGNED, first BIGINT UNSIGNED, last BIGINT UNSIGNED, protocol TINYINT UNSIGNED, flows SMALLINT UNSIGNED, packets MEDIUMINT UNSIGNED, bytes INT UNSIGNED, PRIMARY KEY(`srcaddr`)) \
ENGINE=InnoDB DEFAULT CHARSET=utf8 PARTITION BY HASH(srcaddr) partitions 64;")
mycursor.close()

True

In [3]:
import socket  
import struct  
  

ip = '127.0.0.1'  
int_ip = struct.unpack('!I', socket.inet_aton(ip))[0]  
print(int_ip)  
str_ip = socket.inet_ntoa(struct.pack('!I', int_ip))  
print(str_ip) 


2130706433
127.0.0.1


In [4]:
def protocol_to_number(protocol):
    if protocol == 'TCP':
        num = 1
    elif protocol=='UDP':
        num = 2
    elif protocol == 'ssh':
        num = 3
    return num

In [5]:
# Python read all files from a directory in order
import os

f = open("./data/netflow_name.txt", 'w')
path = "./data/netflow/"
files = os.listdir(path)
files.sort()

s = []

for file_name in files:
    if not os.path.isdir(path + file_name):
        f_name = str(file_name)
        print(f_name)
        s.append(f_name)
        f.write(f_name + '\n')
for i in s:
    print(i)
print(s)

20200528140758.json
20200528141325.json
20200528141851.json
20200528142416.json
20200528142940.json
20200528143504.json
20200528144029.json
20200528144554.json
20200528145119.json
20200528145643.json
20200528150208.json
20200528150732.json
20200528151257.json
20200528151821.json
20200528152346.json
20200528152910.json
20200528153435.json
20200528153959.json
20200528154524.json
20200528155048.json
20200528155613.json
20200528160137.json
20200528160702.json
20200528161228.json
20200528161752.json
20200528162317.json
20200528162841.json
20200528163405.json
20200528163930.json
20200528164454.json
20200528165019.json
20200528165543.json
20200528170107.json
20200528170632.json
20200528171156.json
20200528171721.json
20200528172246.json
20200528172810.json
20200528173335.json
20200528173859.json
20200528174424.json
20200528174948.json
20200528175513.json
20200528180037.json
20200528180602.json
20200528181126.json
20200528181650.json
20200528182215.json
20200528182740.json
20200528183304.json


In [6]:
import time
import json

# update 1 X 1000000 records at a time
# Use List to collect the time of each insert operation for every 1 million records
time_records = []
time_records_opt = []
for json_name in s:
    mydb.reconnect()
    mycursor = mydb.cursor()
    sql_update = "INSERT INTO p_netflow (srcaddr, srcport, first, last, protocol, flows, packets, bytes) VALUES (%s,%s,%s,%s,%s,%s,%s,%s) ON DUPLICATE KEY UPDATE `last` = VALUES(`last`), `flows`= VALUES(`flows`) + `flows`, `packets`= VALUES(`packets`) + `packets`, `bytes`= VALUES(`bytes`) + `bytes`"

    start_time = time.time()
    flows = []
    with open('data/netflow/' + json_name) as record:
        new_record = json.load(record)
        new_record.sort(key=lambda k: (k.get('srcaddr', 0)))
        for flow in new_record:
            srcaddr = struct.unpack('!I', socket.inet_aton(flow["srcaddr"]))[0]
            protocol = protocol_to_number(flow["protocol"])
            flow_one = (srcaddr, flow["srcport"], flow["First"], flow["Last"], protocol, flow["flows"], flow["packets"], flow["bytes"])
            flows.append(flow_one)
        mycursor.executemany(sql_update, flows)
        print(mycursor.rowcount, "records inserted successful。")
        mycursor.close()
        mydb.commit()    # 数据表内容有更新，必须使用到该语句
        mydb.close()
        end_time = time.time()
        consume_time = end_time - start_time
        time_records.append(consume_time)
        print(consume_time) 
        consume_time = '{:0>2s}'.format(str(int(consume_time // 3600))) \
               + ':{:0>2s}'.format(str(int((consume_time // 60) % 60))) \
               + ':{:0>2s}'.format(str(int(consume_time % 60)))
        time_records_opt.append(consume_time)
        print(consume_time) 

1000133 records inserted successful。
73.09140038490295
00:01:13
1000414 records inserted successful。
103.19617938995361
00:01:43
1000623 records inserted successful。
87.90693759918213
00:01:27
1000830 records inserted successful。
106.10269737243652
00:01:46
1001038 records inserted successful。
90.1386251449585
00:01:30
1001239 records inserted successful。
88.74583983421326
00:01:28
1001565 records inserted successful。
87.87718987464905
00:01:27
1001741 records inserted successful。
137.15481877326965
00:02:17
1001998 records inserted successful。
111.45083904266357
00:01:51
1002259 records inserted successful。
105.69331812858582
00:01:45
1002533 records inserted successful。
118.32273054122925
00:01:58
1002801 records inserted successful。
124.2224063873291
00:02:04
1002907 records inserted successful。
129.0814826488495
00:02:09
1003270 records inserted successful。
160.2796812057495
00:02:40


OperationalError: 2055: Lost connection to MySQL server at 'localhost:3306', system error: 32 Broken pipe

## Use different amount of partitions

In [9]:
# partitions = 64
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE p2_netflow (srcaddr INT UNSIGNED, srcport MEDIUMINT UNSIGNED, first BIGINT UNSIGNED, last BIGINT UNSIGNED, protocol TINYINT UNSIGNED, flows SMALLINT UNSIGNED, packets MEDIUMINT UNSIGNED, bytes INT UNSIGNED, PRIMARY KEY(`srcaddr`)) \
ENGINE=InnoDB DEFAULT CHARSET=utf8 PARTITION BY HASH(srcaddr) partitions 64;")
mycursor.close()

True

In [ ]:
import time
import json

# update 1 X 1000000 records at a time
# Use List to collect the time of each insert operation for every 1 million records
time_records = []
time_records_opt = []
for json_name in s:
#    mydb.reconnect()
    mycursor = mydb.cursor()
    sql_update = "INSERT INTO p2_netflow (srcaddr, srcport, first, last, protocol, flows, packets, bytes) VALUES (%s,%s,%s,%s,%s,%s,%s,%s) ON DUPLICATE KEY UPDATE `last` = VALUES(`last`), `flows`= VALUES(`flows`) + `flows`, `packets`= VALUES(`packets`) + `packets`, `bytes`= VALUES(`bytes`) + `bytes`"

    start_time = time.time()
    flows = []
    with open('data/netflow/' + json_name) as record:
        new_record = json.load(record)
        new_record.sort(key=lambda k: (k.get('srcaddr', 0)))
        for flow in new_record:
            srcaddr = struct.unpack('!I', socket.inet_aton(flow["srcaddr"]))[0]
            protocol = protocol_to_number(flow["protocol"])
            flow_one = (srcaddr, flow["srcport"], flow["First"], flow["Last"], protocol, flow["flows"], flow["packets"], flow["bytes"])
            flows.append(flow_one)
        mycursor.executemany(sql_update, flows)
        print(mycursor.rowcount, "records inserted successful。")
        mycursor.close()
        mydb.commit()    # 数据表内容有更新，必须使用到该语句
#        mydb.close()
        end_time = time.time()
        consume_time = end_time - start_time
        time_records.append(consume_time)
        print(consume_time) 
        consume_time = '{:0>2s}'.format(str(int(consume_time // 3600))) \
               + ':{:0>2s}'.format(str(int((consume_time // 60) % 60))) \
               + ':{:0>2s}'.format(str(int(consume_time % 60)))
        time_records_opt.append(consume_time)
        print(consume_time) 
        
mydb.close()

1000133 records inserted successful。
75.32987833023071
00:01:15
1000414 records inserted successful。
71.39967608451843
00:01:11
1000623 records inserted successful。
64.01525068283081
00:01:04
1000830 records inserted successful。
67.0480740070343
00:01:07
1001038 records inserted successful。
68.92629933357239
00:01:08
1001239 records inserted successful。
74.25038123130798
00:01:14
1001565 records inserted successful。
77.28313946723938
00:01:17
1001741 records inserted successful。
82.74110746383667
00:01:22
1001998 records inserted successful。
98.42004656791687
00:01:38
1002259 records inserted successful。
92.67580986022949
00:01:32
1002533 records inserted successful。
99.5465350151062
00:01:39
1002801 records inserted successful。
107.93234872817993
00:01:47
1002907 records inserted successful。
105.80985164642334
00:01:45
1003270 records inserted successful。
118.93058133125305
00:01:58
1003449 records inserted successful。
128.23208665847778
00:02:08
1003593 records inserted successful。
1

In [ ]:
#first replace srcaddr with number then sort it


In [6]:
# 2.partitions = 1000
mycursor = mydb.cursor()
mycursor.execute("drop table p3_netflow")
mycursor.execute("CREATE TABLE p3_netflow (srcaddr INT UNSIGNED, srcport MEDIUMINT UNSIGNED, first BIGINT UNSIGNED, last BIGINT UNSIGNED, protocol TINYINT UNSIGNED, flows SMALLINT UNSIGNED, packets MEDIUMINT UNSIGNED, bytes INT UNSIGNED, PRIMARY KEY(`srcaddr`)) \
ENGINE=InnoDB DEFAULT CHARSET=utf8 PARTITION BY HASH(srcaddr) partitions 1000;")
mycursor.close()

True

In [7]:
import time
import json

# update 1 X 1000000 records at a time
# Use List to collect the time of each insert operation for every 1 million records
time_records = []
time_records_opt = []
for json_name in s:
#    mydb.reconnect()
    mycursor = mydb.cursor()
    sql_update = "INSERT INTO p3_netflow (srcaddr, srcport, first, last, protocol, flows, packets, bytes) VALUES (%s,%s,%s,%s,%s,%s,%s,%s) ON DUPLICATE KEY UPDATE `last` = VALUES(`last`), `flows`= VALUES(`flows`) + `flows`, `packets`= VALUES(`packets`) + `packets`, `bytes`= VALUES(`bytes`) + `bytes`"

    start_time = time.time()
    flows = []
    with open('data/netflow/' + json_name) as record:
        new_record = json.load(record)
        new_record.sort(key=lambda k: (k.get('srcaddr', 0)))
        for flow in new_record:
            srcaddr = struct.unpack('!I', socket.inet_aton(flow["srcaddr"]))[0]
            protocol = protocol_to_number(flow["protocol"])
            flow_one = (srcaddr, flow["srcport"], flow["First"], flow["Last"], protocol, flow["flows"], flow["packets"], flow["bytes"])
            flows.append(flow_one)
        mycursor.executemany(sql_update, flows)
        print(mycursor.rowcount, "records inserted successful。")
        mycursor.close()
        mydb.commit()    # 数据表内容有更新，必须使用到该语句
#        mydb.close()
        end_time = time.time()
        consume_time = end_time - start_time
        time_records.append(consume_time)
        print(consume_time) 
        consume_time = '{:0>2s}'.format(str(int(consume_time // 3600))) \
               + ':{:0>2s}'.format(str(int((consume_time // 60) % 60))) \
               + ':{:0>2s}'.format(str(int(consume_time % 60)))
        time_records_opt.append(consume_time)
        print(consume_time) 
        
mydb.close()

1000133 records inserted successful。
70.53857135772705
00:01:10
1000414 records inserted successful。
116.8901743888855
00:01:56
1000623 records inserted successful。
101.48099136352539
00:01:41
1000830 records inserted successful。
132.86864399909973
00:02:12
1001038 records inserted successful。
300.03582286834717
00:05:00
1001239 records inserted successful。
187.10230350494385
00:03:07
1001565 records inserted successful。
121.22623586654663
00:02:01
1001741 records inserted successful。
114.00706052780151
00:01:54
1001998 records inserted successful。
120.22212839126587
00:02:00
1002259 records inserted successful。
182.42511796951294
00:03:02


KeyboardInterrupt: 

In [8]:
# after close log_err and delete some tables
mycursor = mydb.cursor()
mycursor.execute("drop table p3_netflow")
mycursor.execute("CREATE TABLE p3_netflow (srcaddr INT UNSIGNED, srcport MEDIUMINT UNSIGNED, first BIGINT UNSIGNED, last BIGINT UNSIGNED, protocol TINYINT UNSIGNED, flows SMALLINT UNSIGNED, packets MEDIUMINT UNSIGNED, bytes INT UNSIGNED, PRIMARY KEY(`srcaddr`)) \
ENGINE=InnoDB DEFAULT CHARSET=utf8 PARTITION BY HASH(srcaddr) partitions 1000;")
mycursor.close()

True

In [11]:
import time
import json

# update 1 X 1000000 records at a time
# Use List to collect the time of each insert operation for every 1 million records
time_records = []
time_records_opt = []
for json_name in s:
#    mydb.reconnect()
    mycursor = mydb.cursor()
    sql_update = "INSERT INTO p3_netflow (srcaddr, srcport, first, last, protocol, flows, packets, bytes) VALUES (%s,%s,%s,%s,%s,%s,%s,%s) ON DUPLICATE KEY UPDATE `last` = VALUES(`last`), `flows`= VALUES(`flows`) + `flows`, `packets`= VALUES(`packets`) + `packets`, `bytes`= VALUES(`bytes`) + `bytes`"

    start_time = time.time()
    flows = []
    with open('data/netflow/' + json_name) as record:
        new_record = json.load(record)
        new_record.sort(key=lambda k: (k.get('srcaddr', 0)))
        for flow in new_record:
            srcaddr = struct.unpack('!I', socket.inet_aton(flow["srcaddr"]))[0]
            protocol = protocol_to_number(flow["protocol"])
            flow_one = (srcaddr, flow["srcport"], flow["First"], flow["Last"], protocol, flow["flows"], flow["packets"], flow["bytes"])
            flows.append(flow_one)
        mycursor.executemany(sql_update, flows)
        print(mycursor.rowcount, "records inserted successful。")
        mycursor.close()
        mydb.commit()    # 数据表内容有更新，必须使用到该语句
#        mydb.close()
        end_time = time.time()
        consume_time = end_time - start_time
        time_records.append(consume_time)
        print(consume_time) 
        consume_time = '{:0>2s}'.format(str(int(consume_time // 3600))) \
               + ':{:0>2s}'.format(str(int((consume_time // 60) % 60))) \
               + ':{:0>2s}'.format(str(int(consume_time % 60)))
        time_records_opt.append(consume_time)
        print(consume_time) 
        
mydb.close()

1000133 records inserted successful。
76.20668983459473
00:01:16
1000414 records inserted successful。
113.39053893089294
00:01:53
1000623 records inserted successful。
100.34453392028809
00:01:40
1000830 records inserted successful。
131.0168161392212
00:02:11
1001038 records inserted successful。
367.76259899139404
00:06:07
1001239 records inserted successful。
196.93707990646362
00:03:16
1001565 records inserted successful。
105.03025698661804
00:01:45
1001741 records inserted successful。
105.3196702003479
00:01:45
1001998 records inserted successful。
109.6242208480835
00:01:49
1002259 records inserted successful。
107.96849966049194
00:01:47
1002533 records inserted successful。
130.78641057014465
00:02:10
1002801 records inserted successful。
122.24328517913818
00:02:02
1002907 records inserted successful。
140.43623876571655
00:02:20
1003270 records inserted successful。
158.18153166770935
00:02:38
1003449 records inserted successful。
174.97961115837097
00:02:54
1003593 records inserted succ

DatabaseError: 1114 (HY000): The table 'p3_netflow' is full